In [ ]:
import xarray as xr
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob

In [ ]:
var = "psl"

In [ ]:
root = "/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v2.urban/"
domain= "CORDEX-COREURB" #"CORDEX-CORE"/"CORDEX-EUR-11"
scenario = 'rcp85'
if scenario == 'historical:
    name = f"{var}_{domain}_historical_mon_197001-200512_v02.nc"
    if var in ['cd','hd']:
        name = f'{var}_{domain}_historical_yr_1970-2005_v02.nc'
    elif 'var'== 'sfcwind':
        name = f'sfcwind_{domain}_historical_mon_197001-200512_v02.nc'
else:
    name = f"{var}_{domain}_{scenario}_mon_200601-210012_v02.nc"
    if var in ['cd','hd']:
        name = f'{var}_{domain}_{scenario}_yr_2006-2100_v02.nc'
    elif 'var'== 'sfcwind':
        name = f'sfcwind_{domain}_{scenario}_mon_200601-210012_v02.nc'

In [ ]:
root_nc = root + domain + '/' + scenario + '/' + name

ds_urb = xr.open_dataset(root_nc, engine="h5netcdf", 
                    chunks={"time": 120})

In [ ]:
root = "/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v2.urban/"
domain= "CORDEX-CORERUR" #"CORDEX-CORE"/"CORDEX-EUR-11"
scenario = 'rcp85'
if scenario == 'historical:
    name = f"{var}_{domain}_historical_mon_197001-200512_v02.nc"
    if var in ['cd','hd']:
        name = f'{var}_{domain}_historical_yr_1970-2005_v02.nc'
    elif 'var'== 'sfcwind':
        name = f'sfcwind_{domain}_historical_mon_197001-200512_v02.nc'
else:
    name = f"{var}_{domain}_{scenario}_mon_200601-210012_v02.nc"
    if var in ['cd','hd']:
        name = f'{var}_{domain}_{scenario}_yr_2006-2100_v02.nc'
    elif 'var'== 'sfcwind':
        name = f'sfcwind_{domain}_{scenario}_mon_200601-210012_v02.nc'

In [ ]:
root_nc = root + domain + '/' + scenario + '/' + name
var = name.split('_')[0]

ds_rur = xr.open_dataset(root_nc, engine="h5netcdf", 
                    chunks={"time": 120})

In [ ]:
# Get all shapefiles in the specified directory
shapefiles = glob.glob("box/*.shp")

# Loop through each shapefile to crop and plot datasets
for shp_path in shapefiles:
    city_name = shp_path.split("/")[-1].replace(".shp", "")  # Extract city name from filename
    print(f"Processing: {city_name}")

    # Load the shapefile and convert it to EPSG:4326 projection
    bbox = gpd.read_file(shp_path).to_crs("EPSG:4326")

    # Ensure that the bounding box properly covers the dataset's extent to avoid empty selections
    ds_urb_crop = ds_urb.sel(
        lon=slice(bbox.total_bounds[0], bbox.total_bounds[2]),
        lat=slice(bbox.total_bounds[1], bbox.total_bounds[3])
    )

    ds_rur_crop = ds_rur.sel(
        lon=slice(bbox.total_bounds[0], bbox.total_bounds[2]),
        lat=slice(bbox.total_bounds[1], bbox.total_bounds[3])
    )

    # Create subplots for urban and rural datasets
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={"projection": ccrs.PlateCarree()})

    for ax, ds, title in zip(axes, [ds_urb_crop, ds_rur_crop], ["Urban", "Rural"]):
        # Compute the mean across "member" and "time" dimensions
        da = ds[var].mean(dim=["member", "time"], skipna=True)

        # Plot the data
        im = da.plot(ax=ax, transform=ccrs.PlateCarree())

        # Add map features
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=":")
        
        # Check if 'bbox.boundary' exists before plotting
        bbox.boundary.plot(ax=ax, edgecolor="black")  
        
        ax.set_title(f"{title} - {city_name}")

    plt.show()